In [2]:
import torch

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import butter, lfilter , filtfilt
import pandas as pd
import scipy as sp
import os
import sys
import seaborn as sb 
from matplotlib import cm

import pickle


# Load the Dic

In [4]:

class DfSubject:
    def __init__(self, elec_i, muscle_i):
        self.Elec_config=list_elecs[elec_i]
        self.List_elecs=list_elecs
        self.Muscle_or_other=list_muscle_or_other_elec_i[muscle_i]
        self.List_muscle_or_other=list_muscle_or_other_elec_i
        self.Trial_number= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Trial_number"]
        self.Trial_number=[int(i) for i in self.Trial_number] #save as int
        self.Raw= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Raw"]
        self.Raw.index=self.Trial_number
        self.Time_points= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Time_points"]
        self.Time_points.index=self.Trial_number
        self.Amplitudes= df_subject_no_empty[(df_subject_no_empty["Elec_config"]==self.Elec_config)& (df_subject_no_empty["Muscle_or_other"]== self.Muscle_or_other)]["Amplitudes"]
        self.Amplitudes.index=self.Trial_number
        self.TS_arts_1="$"
        
        # self.TS_arts_1.index=self.Trial_number
        self.Time_points_arts_1=-1
        self.TS_arts_2="$"
        self.Time_points_arts_2=-1
        self.TS_arts_3="$"
        self.Time_points_arts_3=-1


# Goal is to chose an electrode and a muscle and see all the trials and the corrected data        
#  Access data the following way: pData["_3_5"].Raw[19] #prints the raw data from trial 19

In [5]:
### retrieve  dfSubject structured as pData  #
#run the structure of the data before opening the pickle (wihtout creating an object)
# command pData["_0_"].Trial_number=0
import pickle
run=1
if run == 1:  

    with open('Class2_Dump.pkl', 'rb') as handle:
        pData = pickle.load(handle)


# Make a tensor

Always retrieve pData["_0_0"] since some list_elec and list_muscle is accesced from there

Build the tensor matrix with the raw data.
Time X Electrodes X Muscles X Amplitudes (Stimulation) X Muscle activity (EMG)


In [6]:
device =  "cuda" if torch.cuda.is_available() else "cpu"

## Try to do it with class

### Extract list

The 5 dimensions that should be included in the tensor 
    
    elecs_to_nr, # pandas df
    muscles_to_nr # pandas df
    raw_data_to_matrix, 
    amplitudes_to_list, 
    time_points_to_list

In [7]:

## save Amplitudes in a list ##
amplitudes_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                amplitudes_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Amplitudes[trial])
                

print(len(amplitudes_to_list)) #16410

16410


In [8]:

## save raw_data in a list ##
#with append() in order to save lists of lists
raw_data_to_matrix=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                raw_data_to_matrix.append(pData[f'_{elec_i}_{muscle_i}'].Raw[trial])
               

print(len(raw_data_to_matrix))
print((raw_data_to_matrix[0].dtype)) #float64
# print((raw_data_to_matrix[0:10].dtype)) #error


16410
float64


In [9]:

## save the Time_points in a list ##
time_points_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                time_points_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Time_points[trial])
                

print(len(time_points_to_list)) #16410


16410


In [10]:
## save Muscles in a list ##
muscles_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                muscles_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Muscle_or_other)
                

print(len(muscles_to_list)) #16410
print(muscles_to_list[0:10])

16410
['LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST', 'LST']


In [11]:
## save Elecs in a list ##
elecs_to_list=[]
for elec_i in range (len(pData["_0_0"].List_elecs)) : #loop through all the elecs
    for muscle_i in range (len(pData["_0_0"].List_muscle_or_other)): # loop through all the muscles
            for trial in pData[f'_{elec_i}_{muscle_i}'].Trial_number: #loop through the trials
                elecs_to_list.append(pData[f'_{elec_i}_{muscle_i}'].Elec_config)
                

print(len(elecs_to_list)) #16410
print(elecs_to_list[0:10])

16410
['Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_', 'Elec01_']


#### Use Hot encoder to convert from str to number
for muscles and elecs (since the data is str and for the tensor can only have one data type)

In [12]:
# Use hot encoder to convert muscles to nrs
import pandas as pd
muscles_to_nr=pd.get_dummies(
    np.array(muscles_to_list), 
    prefix=None, 
    prefix_sep='_', 
    dummy_na=False, 
    columns=None, 
    sparse=False, 
    drop_first=False, 
    dtype=None
)

In [13]:
# Use hot encoder to convert elecs to nrs
import pandas as pd
elecs_to_nr=pd.get_dummies(
    np.array(elecs_to_list), 
    prefix=None, 
    prefix_sep='_', 
    dummy_na=False, 
    columns=None, 
    sparse=False, 
    drop_first=False, 
    dtype=None
)

### Save the data in a new tensor

elecs_to_nr, # pandas df
muscles_to_nr # pandas df
raw_data_to_matrix, 
amplitudes_to_list, 
time_points_to_list

#### concatenate data to one df

In [23]:
##### concat muscles and elecs ###
muscles_to_nr.reset_index(drop=True, inplace=True)
elecs_to_nr.reset_index(drop=True, inplace=True)
muscles_to_nr.index = elecs_to_nr.index
muscles_and_elecs=pd.concat([muscles_to_nr, elecs_to_nr], axis=1)

##### add the raw data
raw_data_to_df = pd.DataFrame(raw_data_to_matrix) #convert to a pandas df
if len(raw_data_to_df.iloc[0])!=len((raw_data_to_matrix[0])): #get rid of nan column
    raw_data_to_df=raw_data_to_df.iloc[: , :-1]
raw_data_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs.reset_index(drop=True, inplace=True)
muscles_and_elecs.index = raw_data_to_df.index
muscles_and_elecs_and_raw=pd.concat([muscles_and_elecs, raw_data_to_df], axis=1)

##### add the amplitude
amplitudes_to_df=pd.DataFrame(amplitudes_to_list) #convert to a pandas df
amplitudes_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw.index = amplitudes_to_df.index
muscles_and_elecs_and_raw_and_amplitudes=pd.concat([muscles_and_elecs_and_raw, amplitudes_to_df], axis=1)

##### add time_points
time_points_to_df=pd.DataFrame(time_points_to_list) #convert to a pandas df
time_points_to_df.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw_and_amplitudes.reset_index(drop=True, inplace=True)
muscles_and_elecs_and_raw_and_amplitudes.index = time_points_to_df.index
muscles_and_elecs_and_raw_and_amplitudes_and_time_points=pd.concat([muscles_and_elecs_and_raw_and_amplitudes, time_points_to_df], axis=1)
# print(pd.concat([muscles_and_elecs_and_raw_and_amplitudes[0:10], time_points_to_df[0:10]], axis=1))


#### convert df to tensor

In [22]:
#### Convert df to tensor ###
# determine the supported device
#with correction of nan values
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(muscles_and_elecs_and_raw_and_amplitudes_and_time_points):
    device = get_device()
    return torch.from_numpy(muscles_and_elecs_and_raw_and_amplitudes_and_time_points.values).float().to(device)

df_tensor = df_to_tensor(muscles_and_elecs_and_raw_and_amplitudes_and_time_points)
# series_tensor = df_to_tensor(series)
print(df_tensor[0:10])
print(df_tensor[0:10].shape)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.5000e+00,
         1.4030e+03],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.0098e-01, 1.7000e+00,
         1.4030e+03]])
torch.Size([10, 1440])


# Save the data as a pickle file